# Smartphone

In [25]:
import pandas as pd
import nltk
import xmltodict
from dicttoxml import dicttoxml
from xml.dom.minidom import parseString

In [26]:
%run ../funcoes.py

Loading functions...


In [27]:
# carregando tipos
domain = "smartphone"
tiposAll = pd.read_csv('../data/tipologia/categorias_iacs.csv', sep=',', quoting=0, keep_default_na=False)
tipos = tiposAll.loc[tiposAll['domain'] == domain]

In [4]:
dfTest = pd.read_csv('../07_Keras/data/domain_smartphone_IOB_test.csv', sep='\t', quoting=3, keep_default_na=False)
dfTrain = pd.read_csv('../07_Keras/data/domain_smartphone_IOB_train.csv', sep='\t', quoting=3, keep_default_na=False)

In [5]:
f = open("../07_Keras/data/aspectsTest.xml","r")
xml = f.read()
aspectsTest = xmltodict.parse(xml, process_namespaces=True)

In [6]:
def result2aspecList(y_pred, X_test):
    aspectsList = []
    aspect = []
    stopwords = nltk.corpus.stopwords.words('portuguese')
    for sent, sent_pred in zip(X_test, y_pred):
        #print(len(sent),len(sent_pred))
        for token, tag in zip(sent, sent_pred):
            if tag != 'O':
                #print(tag, token)
                #print(tag, token['word.lower()'])
                if tag == 'B-asp' and aspect:
                    auxAspect = " ".join(aspect).lower()
                    if (auxAspect not in stopwords and len(auxAspect) > 1):
                        aspectsList.append(auxAspect)
                    aspect = []
                if token not in [',',':']:
                    aspect.append(token.lower())

    aspectsList = list(set(aspectsList))
    aspectsList.sort()    
    return aspectsList

In [7]:
tag_scheme = [
'B-asp',
'I-asp'
]

In [8]:
contSents = 0
for count, row in dfTrain.iterrows():
    if (row[0] != "" and count > 0):
        contSents = contSents + 1
contSents = contSents + 1

In [9]:
contTrain = round(contSents * 0.9)
print(contTrain)

226


In [10]:
training = {'sentences': [], 'tags' : []}
validation = {'sentences': [], 'tags' :[]}
sent = []
tags = []
contSents = 0
for count, row in dfTrain.iterrows():
    if (row[0] != "" and count > 0):
        if contSents < contTrain:
            training['sentences'].append(sent)
            training['tags'].append(tags)
        else:
            validation['sentences'].append(sent)
            validation['tags'].append(tags)

        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
        
validation['sentences'].append(sent)
validation['tags'].append(tags)

In [11]:
len(training['sentences'])

226

In [12]:
len(validation['sentences'])

25

In [13]:
test = {'sentences': [], 'tags' : []}
sent = []
tags = []
contSents = 0
for count, row in dfTest.iterrows():
    if (row[0] != "" and count > 0):
        test['sentences'].append(sent)
        test['tags'].append(tags)
        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
test['sentences'].append(sent)
test['tags'].append(tags)

In [14]:
len(test['sentences'])

106

In [15]:
transformer = 'bert-base-multilingual-uncased'

In [16]:
# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
    'epochs' : 40,
    'warmup_steps' : 500,                                                   
    'train_batch_size': 13,
    'learning_rate': 0.0001
}

In [18]:
from NERDA.models import NERDA
model = NERDA(
    dataset_training = training,
    dataset_validation = validation,
    tag_scheme = tag_scheme, 
    tag_outside = 'O',
    transformer = transformer,
    dropout = dropout,
    hyperparameters = training_hyperparameters
)

Device automatically set to: cpu


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
model.train()


 Epoch 1 / 40


 50%|██████████████████████████████████████████████████████████████████████████████████████▌                                                                                      | 9/18 [01:25<01:23,  9.30s/it]/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #149 length 128 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.85s/it]


Train Loss = 0.944373776515325 Valid Loss = 0.45442693680524826

 Epoch 2 / 40


 50%|██████████████████████████████████████████████████████████████████████████████████████▌                                                                                      | 9/18 [01:30<01:26,  9.59s/it]/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #149 length 128 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.87s/it]


Train Loss = 0.4077020262678464 Valid Loss = 0.3394068256020546

 Epoch 3 / 40


...

 50%|██████████████████████████████████████████████████████████████████████████████████████▌                                                                                      | 9/18 [00:47<00:45,  5.08s/it]/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #149 length 128 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.13s/it]


Train Loss = 0.0002208370206062682 Valid Loss = 0.31072520289853856

 Epoch 40 / 40


 50%|██████████████████████████████████████████████████████████████████████████████████████▌                                                                                      | 9/18 [00:47<00:45,  5.06s/it]/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #149 length 128 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.08s/it]

Train Loss = 0.0007504083809585103 Valid Loss = 0.31072520289853856


'Model trained successfully'

In [20]:
model.evaluate_performance(test)

/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 23 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded lengths of predicted values in {n_exceeds} cases and were truncated. _Consider_ increasing max_len parameter for your model.')
/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 23 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded lengths of predicted values in {n_exceeds} cases and were truncated. _Consider_ increasing max_len parameter for your model.')
/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/douto

,Level,F1-Score,Precision,Recall
0,B-asp,0.463277,0.460674,0.465909
1,I-asp,0.404255,0.431818,0.380000
0,AVG_MICRO,0.450893,NaN,NaN
0,AVG_MICRO,0.433766,NaN,NaN


In [21]:
y_pred = model.predict(test['sentences'])

In [22]:
aspects = result2aspecList(y_pred, test['sentences'])
res = calculaMetricas(aspects, aspectsTest['datasets']['smartphone'], lemma=False, output=False)
res

{'totalAspects': 158,
 'pred': 85,
 'tp': 75,
 'fp': 46,
 'fn': 83,
 'precisao': 0.6198347107438017,
 'precisaoLaplace': 0.6178861788617886,
 'revocacao': 0.47468354430379744,
 'medidaf': 0.5376344086021505,
 'totalExplicitos': 134,
 'explicitosOk': 35,
 'pExplicitos': 0.26119402985074625,
 'totalImplicitos': 24,
 'implicitosOk': 4,
 'pImplicitos': 0.16666666666666666}

In [28]:
aspects = result2aspecList(y_pred, test['sentences'])
res = calculaMetricas(
    aspects, 
    aspectsTest['datasets'][domain], 
    tipos,
    lemma=False, 
    output=False
)

xml = dicttoxml(res, attr_type=False, custom_root='resultados')
with open('../resultados/bert_multi_indomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())    
res

{'totalAspects': 158,
 'pred': 85,
 'tp': 75,
 'fp': 46,
 'fn': 83,
 'precisao': 0.6198347107438017,
 'precisaoLaplace': 0.6178861788617886,
 'revocacao': 0.47468354430379744,
 'medidaf': 0.5376344086021505,
 'totalExplicitos': 134,
 'explicitosOk': 35,
 'pExplicitos': 0.26119402985074625,
 'explicitsList': ['produto',
  'touch',
  'funções',
  'menu',
  'smartphone',
  'wifi',
  'recursos',
  'gps',
  'bateria',
  'wi-fi',
  'visor',
  'fotografia',
  'aparelho',
  'luz de flash',
  'botões',
  'celular',
  'memoria',
  'camera digital',
  'preço',
  'volume de áudio',
  'display',
  'camera',
  'internet',
  'nokia',
  'câmera',
  'radio',
  'funcionalidades',
  'qualidade sonora',
  'custo-benefício',
  'fabricante',
  'carregador de carro',
  'marca',
  'rádio',
  'modelo',
  'e-mail'],
 'totalImplicitos': 24,
 'implicitosOk': 4,
 'pImplicitos': 0.16666666666666666,
 'implicitsList': ['trava', 'prático', 'pequeno', 'fácil de mexer'],
 'tipos': {'Event': {'Verb': 3}, 'Qualification'

## Bert Portuguese: Bertimbau

In [29]:
transformer = 'neuralmind/bert-base-portuguese-cased'

# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
    'epochs' : 40,
    'warmup_steps' : 500,                                                   
    'train_batch_size': 13,
    'learning_rate': 0.0001
}

from NERDA.models import NERDA
model2 = NERDA(
    dataset_training = training,
    dataset_validation = validation,
    tag_scheme = tag_scheme, 
    tag_outside = 'O',
    transformer = transformer,
    dropout = dropout,
    hyperparameters = training_hyperparameters
)

Device automatically set to: cpu


Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
model2.train()


 Epoch 1 / 40


 50%|██████████████████████████████████████████████████████████████████████████████████████▌                                                                                      | 9/18 [02:58<01:03,  7.01s/it]/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #149 length 131 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.27s/it]


Train Loss = 1.0293202035956912 Valid Loss = 0.737432524561882

 Epoch 2 / 40


 50%|██████████████████████████████████████████████████████████████████████████████████████▌                                                                                      | 9/18 [00:43<00:41,  4.64s/it]/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #149 length 131 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.08s/it]


Train Loss = 0.5076665927966436 Valid Loss = 0.3023553416132927

 Epoch 3 / 40


...

 50%|██████████████████████████████████████████████████████████████████████████████████████▌                                                                                      | 9/18 [00:44<00:43,  4.85s/it]/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #149 length 131 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.15s/it]


Train Loss = 0.0003647147297872127 Valid Loss = 0.2109511349353852

 Epoch 40 / 40


 50%|██████████████████████████████████████████████████████████████████████████████████████▌                                                                                      | 9/18 [00:45<00:44,  4.92s/it]/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #149 length 131 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.09s/it]


Train Loss = 0.0005864600866011137 Valid Loss = 0.2109511349353852


'Model trained successfully'

In [31]:
model2.evaluate_performance(test)

/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 23 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded lengths of predicted values in {n_exceeds} cases and were truncated. _Consider_ increasing max_len parameter for your model.')
/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 23 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded lengths of predicted values in {n_exceeds} cases and were truncated. _Consider_ increasing max_len parameter for your model.')
/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/douto

,Level,F1-Score,Precision,Recall
0,B-asp,0.440111,0.431694,0.448864
1,I-asp,0.340000,0.340000,0.340000
0,AVG_MICRO,0.418301,NaN,NaN
0,AVG_MICRO,0.390056,NaN,NaN


In [32]:
y_pred = model2.predict(test['sentences'])

In [33]:
aspects = result2aspecList(y_pred, test['sentences'])
res = calculaMetricas(
    aspects, 
    aspectsTest['datasets'][domain], 
    tipos,
    lemma=False, 
    output=False
)
xml = dicttoxml(res, attr_type=False, custom_root='resultados')
with open('../resultados/bert_bertimbau_indomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())    
res

{'totalAspects': 158,
 'pred': 101,
 'tp': 78,
 'fp': 60,
 'fn': 80,
 'precisao': 0.5652173913043478,
 'precisaoLaplace': 0.5642857142857143,
 'revocacao': 0.4936708860759494,
 'medidaf': 0.527027027027027,
 'totalExplicitos': 134,
 'explicitosOk': 34,
 'pExplicitos': 0.2537313432835821,
 'explicitsList': ['produto',
  'touch',
  'funções',
  'menu',
  'smartphone',
  'recurso',
  'desing',
  'recursos',
  'tamanho',
  'gps',
  'bateria',
  'invetimento',
  'visor',
  'fotografia',
  'aparelho',
  'celular',
  'memoria',
  'camera digital',
  'sd de memória',
  'preço',
  'volume de áudio',
  'display',
  'camera',
  'internet',
  'nokia',
  'câmera',
  'funcionalidades',
  'custo-benefício',
  'fabricante',
  'carregador de carro',
  'marca',
  'designer',
  'modelo',
  'durabilidade'],
 'totalImplicitos': 24,
 'implicitosOk': 7,
 'pImplicitos': 0.2916666666666667,
 'implicitsList': ['trava',
  'congela',
  'prático',
  'restarta',
  'pequeno',
  'lindo',
  'fácil de mexer'],
 'tipos'